In [6]:
from pyspark import SparkContext

# Inicializar SparkContext
sc = SparkContext("local", "SSSP")

23/06/24 18:41:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
# Función para inicializar el grafo con distancias iniciales
def initialize_graph(nodes, source):
    distances = [(node, float('inf')) for node in nodes]
    distances[source] = (source, 0.0)
    return sc.parallelize(distances)

# Función para propagar las distancias a los nodos vecinos
def propagate_distances(node, neighbors, distance):
    propagated_distances = [(neighbor, distance + weight) for neighbor, weight in neighbors]
    propagated_distances.append((node, distance))  # Mantener la distancia actual del nodo
    return propagated_distances

# Función para actualizar las distancias en cada iteración
def update_distances(distances, propagated_distances):
    return distances.union(propagated_distances).reduceByKey(min)

In [8]:
# Grafo de ejemplo con pesos en las aristas
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, [(1, 2), (2, 4)]), (1, [(0, 2), (2, 1), (3, 5)]), (2, [(0, 4), (1, 1), (4, 3)]),
         (3, [(1, 5), (4, 1), (5, 2)]), (4, [(2, 3), (3, 1), (5, 6)]), (5, [(3, 2), (4, 6)])]

# Nodo fuente
source_node = 0

In [9]:
# Paso 1: Inicializar el grafo con distancias iniciales
distances = initialize_graph(nodes, source_node)

# Iterar el algoritmo hasta que se alcance la convergencia
while True:
    # Paso 2: Propagar las distancias a los nodos vecinos
    propagated_distances = sc.parallelize(edges).join(distances).flatMap(
        lambda x: propagate_distances(x[0], x[1][0], x[1][1]))

    # Paso 3: Actualizar las distancias
    new_distances = update_distances(distances, propagated_distances)

    # Comprobar convergencia
    if new_distances.subtract(distances).count() == 0:
        break

    distances = new_distances

# Resultado final: cada nodo junto con su distancia mínima desde el nodo fuente
result = distances.collect()
for node, distance in result:
    print("Node:", node, "Distance:", distance)

# Detener SparkContext
sc.stop()


Node: 0 Distance: 0.0
Node: 1 Distance: 2.0
Node: 2 Distance: 3.0
Node: 3 Distance: 7.0
Node: 4 Distance: 6.0
Node: 5 Distance: 9.0
